# deployment steps:
0) 
1) make sure that Dockerfile, requirements.txt and main.py is in repo
2) export vars to ~/.zshrc
!!!full paths to the credentials.json and token.json this is for local run
change here GOOGLE_CLOUD_PROJECT, GOOGLE_OAUTH_CLIENT_FILE, GOOGLE_OAUTH_TOKEN_FILE
```
export GOOGLE_GENAI_USE_VERTEXAI=True
export GOOGLE_CLOUD_PROJECT=personalplanner-476218       
export GOOGLE_CLOUD_LOCATION=us-central1
export GOOGLE_OAUTH_CLIENT_FILE="/Users/gman/Documents/hackathon/personalPlanner/.creds/credentials.json"  
export GOOGLE_OAUTH_TOKEN_FILE="/Users/gman/Documents/hackathon/personalPlanner/.creds/token.json"
export MODEL=gemini-2.5-flash
```
3) ```source ~/.zshrc```
NOTE to run the current verison of setup run with python main.py. the above exports are for local run only, not cloudrun!!!

4) deploy containers ( change project name in 2 places here)

```
gcloud run deploy project-planner-service \
  --source . \
  --region us-central1 \
  --project personalplanner-476218 \
  --allow-unauthenticated \
  --set-env-vars=GOOGLE_GENAI_USE_VERTEXAI=True,GOOGLE_CLOUD_PROJECT=personalplanner-476218,GOOGLE_CLOUD_LOCATION=us-central1,MODEL=gemini-2.5-flash,GOOGLE_OAUTH_CLIENT_FILE=/app/.creds/credentials.json,GOOGLE_OAUTH_TOKEN_FILE=/app/.creds/token.json


```

5) run this ( Change project name )
```
gcloud run services describe project-planner-service \
  --region=us-central1 \
  --project=personalplanner-476218 \
  --format='value(spec.template.spec.serviceAccountName)'


```


it returns something like xxxxxxx-compute@developer.gserviceaccount.com
then run what's below with that thingy as your SA_EMAIL
```
SA_EMAIL="98380938461-compute@developer.gserviceaccount.com"      

gcloud projects add-iam-policy-binding personalplanner-476218 \
  --member="serviceAccount:${SA_EMAIL}" \
  --role="roles/aiplatform.user"

```



# Paste your service URL here 

In [1]:
import requests
import json
# Service [project-planner-service] revision [project-planner-service-00002-fhq] has been deployed and is serving 100 percent of traffic.
# Service URL: https://project-planner-service-98380938461.us-central1.run.app
SERVICE_URL = "https://project-planner-service-98380938461.us-central1.run.app"

# Helpers

In [2]:
APP_NAME = "orchestrator"  # from /list-apps
USER_ID = "grant"          # anything you like
SESSION_ID = "sess1"       # anything you like
def pretty_print(obj):
    print(json.dumps(obj, indent=2, ensure_ascii=False))


# 2) List available apps
def list_apps():
    url = f"{SERVICE_URL}/list-apps"
    resp = requests.get(url)
    resp.raise_for_status()
    apps = resp.json()
    print("Available apps:")
    pretty_print(apps)
    return apps


# 3) Create or update a session

def create_session(initial_state=None):
    if initial_state is None:
        initial_state = {}

    url = f"{SERVICE_URL}/apps/{APP_NAME}/users/{USER_ID}/sessions/{SESSION_ID}"
    resp = requests.post(url, json=initial_state)

    print("Status:", resp.status_code)
    print("Headers:", resp.headers)
    print("Body:\n", resp.text)

    # Only raise if it's 4xx/5xx *after* we print
    resp.raise_for_status()

    try:
        session = resp.json()
        print("\nParsed session JSON:")
        pretty_print(session)
        return session
    except Exception as e:
        print("\nJSON parse error:", e)
        return None



# 4) Get session (optional, for debugging)
def get_session():
    url = f"{SERVICE_URL}/apps/{APP_NAME}/users/{USER_ID}/sessions/{SESSION_ID}"
    resp = requests.get(url)
    resp.raise_for_status()
    session = resp.json()
    print("Current session:")
    pretty_print(session)
    return session


# 5) Run the orchestrator with a user message
def run_orchestrator(message_text: str):
    url = f"{SERVICE_URL}/run"
    payload = {
        "app_name": APP_NAME,
        "user_id": USER_ID,
        "session_id": SESSION_ID,
        "new_message": {
            "role": "user",
            "parts": [
                {"text": message_text}
            ]
        }
    }
    resp = requests.post(url, json=payload)
    print("Status:", resp.status_code)
    print("Body:\n", resp.text)
    resp.raise_for_status()
    events = resp.json()
    print("Events from orchestrator:")
    pretty_print(events)
    return events


# 6) Convenience helper: full flow in one call
def send_to_orchestrator(message_text: str, init_if_needed: bool = True):
    """
    - Optionally creates/updates a session first
    - Sends a message
    - Returns events
    """
    if init_if_needed:
        # you can attach any initial session state here if you want
        create_session(initial_state={})

    return run_orchestrator(message_text)


# 7) Delete session (if you want to reset state)
def delete_session():
    url = f"{SERVICE_URL}/apps/{APP_NAME}/users/{USER_ID}/sessions/{SESSION_ID}"
    resp = requests.delete(url)
    if resp.status_code not in (200, 204):
        print("Delete returned:", resp.status_code, resp.text)
    else:
        print("Session deleted.")



In [3]:
apps = list_apps()

Available apps:
[
  "TESTING_apollo_service",
  "calendar_service",
  "gmail_service",
  "google_docs_service",
  "google_drive_service",
  "google_search_service",
  "google_sheets_service",
  "jobs_service",
  "orchestrator",
  "resume_customization",
  "ui",
  "utils"
]


In [4]:
create_session(initial_state={"note": "session created from Jupyter"})


Status: 200
Headers: {'content-type': 'application/json', 'x-cloud-trace-context': 'ea2542ff400858423b595ac962e3ec03;o=1', 'date': 'Sun, 09 Nov 2025 06:59:09 GMT', 'server': 'Google Frontend', 'Content-Length': '146', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'}
Body:
 {"id":"sess1","appName":"orchestrator","userId":"grant","state":{"note":"session created from Jupyter"},"events":[],"lastUpdateTime":1762671549.0}

Parsed session JSON:
{
  "id": "sess1",
  "appName": "orchestrator",
  "userId": "grant",
  "state": {
    "note": "session created from Jupyter"
  },
  "events": [],
  "lastUpdateTime": 1762671549.0
}


{'id': 'sess1',
 'appName': 'orchestrator',
 'userId': 'grant',
 'state': {'note': 'session created from Jupyter'},
 'events': [],
 'lastUpdateTime': 1762671549.0}

In [6]:
events = run_orchestrator(
    """can you make a fodler in google drive called ADK-documents and inside there create a folders ADK-resumes, ADK-projects and one empty google sheet called ADK-links """
)


Status: 500
Body:
 Internal Server Error


HTTPError: 500 Server Error: Internal Server Error for url: https://project-planner-service-98380938461.us-central1.run.app/run

In [ ]:
# add resumes to drive files 
# collect the files when you drop it

# OnClick -> 
(events = run_orchestrator(
    "hey add those files to google drive folder called Resumes with id {}"
)
)
# add projects to drive
# OnClick -> 
(events = run_orchestrator(
    "hey add those files to google drive folder called "projects" with id {}"
)
)
if failed





TypeError: list indices must be integers or slices, not str